In [1]:
# import dependencies
import pandas as pd
import json
import matplotlib.pyplot as plt
import csv, os

from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

# Output File (CSV)
output_data_file = "output_data/US_youtube_trending_video_data.csv"

# %matplotlib notebook
#pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [2]:
#Read CSV
Youtube_Data = pd.read_csv('US_youtube_trending_data_cleaned.csv')

# Convert CSV to Data Frame
Youtube_Data = pd.DataFrame(Youtube_Data)

# preview Data Frame
Youtube_Data.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,view_count,likes,dislikes
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,1514614,156908,5855
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,2381688,146739,2794
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,2038853,353787,2628
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,496771,23251,1856
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,1123889,45802,964


In [3]:
Youtube_Data.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'view_count', 'likes', 'dislikes'],
      dtype='object')

In [4]:
# Observe Data Types
Youtube_Data.dtypes

video_id         object
title            object
publishedAt      object
channelId        object
channelTitle     object
categoryId        int64
trending_date    object
view_count        int64
likes             int64
dislikes          int64
dtype: object

In [5]:
# Number of Records in data
len(Youtube_Data)

143390

In [6]:
print ("Rows     : " , Youtube_Data.shape[0]) #Displays numbers of rows .
print ("Columns  : " , Youtube_Data.shape[1]) # and column our dataset contains.
print ("\nFeatures : \n", Youtube_Data.columns.tolist())#displays column names
print ("\nMissing values :  ", Youtube_Data.isnull().sum().values.sum())   #find missing values
print ("\nUnique values :  \n", Youtube_Data.nunique())  # Count distinct observations

Rows     :  143390
Columns  :  10

Features : 
 ['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle', 'categoryId', 'trending_date', 'view_count', 'likes', 'dislikes']

Missing values :   0

Unique values :  
 video_id          25976
title             26896
publishedAt       25881
channelId          6065
channelTitle       6175
categoryId           15
trending_date       697
view_count       139730
likes             98006
dislikes          13179
dtype: int64


In [7]:
# finding number of null records for each column
Youtube_Data.isna().sum()

video_id         0
title            0
publishedAt      0
channelId        0
channelTitle     0
categoryId       0
trending_date    0
view_count       0
likes            0
dislikes         0
dtype: int64

In [8]:
# Renaming columns
Youtube_Data = Youtube_Data.rename(columns={'likes':'Likes','dislikes':'Dislikes',
                                            'channelTitle':'Channel Title','channelTitle':'Channel Title',
                                           'video_id':'Video ID','channelId':'Channel ID'})

In [9]:
Youtube_Data.head()

,Video ID,title,publishedAt,Channel ID,Channel Title,categoryId,trending_date,view_count,Likes,Dislikes
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,1514614,156908,5855
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,2381688,146739,2794
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,2038853,353787,2628
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,496771,23251,1856
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,1123889,45802,964


In [10]:
# add new column that would contain latest view_count for the particular video.
#since number of views keep on increasing everyday , we will be using only single observation against a video id.
Youtube_Data['count_max_view'] = Youtube_Data.groupby(['Video ID'])['view_count'].transform(max)

In [11]:
# Renaming Data Frame 
Youtube_Unique_Video_Id = Youtube_Data

In [12]:
#changing published date , trending_date to datetime datatype.
Youtube_Unique_Video_Id.publishedAt= pd.to_datetime(Youtube_Unique_Video_Id.publishedAt)
Youtube_Unique_Video_Id.trending_date= pd.to_datetime(Youtube_Unique_Video_Id.trending_date)

In [13]:
#Adding column for year and month to the dataframe
Youtube_Unique_Video_Id['Year']=Youtube_Unique_Video_Id['publishedAt'].dt.year
Youtube_Unique_Video_Id['Month']=Youtube_Unique_Video_Id.publishedAt.dt.month
Youtube_Unique_Video_Id['MonYear']=Youtube_Unique_Video_Id['Month'].astype(str)+ Youtube_Unique_Video_Id['Year'].astype(str)

In [14]:
Youtube_Unique_Video_Id.head(10)


,Video ID,title,publishedAt,Channel ID,Channel Title,categoryId,trending_date,view_count,Likes,Dislikes,count_max_view,Year,Month,MonYear
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12 00:00:00+00:00,1514614,156908,5855,3958226,2020,8,82020
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11 17:00:10+00:00,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12 00:00:00+00:00,2381688,146739,2794,3482753,2020,8,82020
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11 16:34:06+00:00,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12 00:00:00+00:00,2038853,353787,2628,3490530,2020,8,82020
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11 16:38:55+00:00,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12 00:00:00+00:00,496771,23251,1856,1212499,2020,8,82020
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11 15:10:05+00:00,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12 00:00:00+00:00,1123889,45802,964,2400821,2020,8,82020
5,w-aidBdvZo8,I Haven't Been Honest About My Injury.. Here's...,2020-08-11 20:00:04+00:00,UC5zJwsFtEs9WYe3A76p7xIA,Professor Live,24,2020-08-12 00:00:00+00:00,949491,77487,746,2192342,2020,8,82020
6,uet14uf9NsE,OUR FIRST FAMILY INTRO!!,2020-08-12 00:17:41+00:00,UCDSJCBYqL7VQrlXfhr1RtwA,Les Do Makeup,26,2020-08-12 00:00:00+00:00,470446,47990,440,1105873,2020,8,82020
7,ua4QMFQATco,CGP Grey was WRONG,2020-08-11 17:15:11+00:00,UC2C_jShtL725hvbm1arSV9w,CGP Grey,27,2020-08-12 00:00:00+00:00,1050143,89190,854,1670954,2020,8,82020
8,SnsPZj91R7E,SURPRISING MY DAD WITH HIS DREAM TRUCK!! | Lou...,2020-08-10 22:26:59+00:00,UCZDdF_p-L88NWVpzF0vjvMQ,Louie's Life,24,2020-08-12 00:00:00+00:00,1402687,95694,2158,1807840,2020,8,82020
9,SsWHMAhshPQ,Ovi x Natanael Cano x Aleman x Big Soto - Veng...,2020-08-11 23:00:10+00:00,UC648rgJOboZlgcDbW00vTSA,Rancho Humilde,10,2020-08-12 00:00:00+00:00,741028,113983,4373,4903099,2020,8,82020


In [15]:
# make a new dataframe with only single observations for each id.
Youtube_Unique_Video_Id = Youtube_Data.loc[Youtube_Data.view_count == Youtube_Data.count_max_view]
# Youtube_Unique_Video_Id.loc[Youtube_Unique_Video_Id.Year == 2021, 'title'].count()

In [16]:
# Use Youtube_Unique_Video_Id for main dataset

#Youtube_Unique_Video_Id.to_csv(output_data_file)
Youtube_Unique_Video_Id.count()

Video ID          26062
title             26062
publishedAt       26062
Channel ID        26062
Channel Title     26062
categoryId        26062
trending_date     26062
view_count        26062
Likes             26062
Dislikes          26062
count_max_view    26062
Year              26062
Month             26062
MonYear           26062
dtype: int64

In [17]:
# Number of unique records in each column
Youtube_Unique_Video_Id.nunique()

Video ID          25976
title             25849
publishedAt       25593
Channel ID         6037
Channel Title      6131
categoryId           15
trending_date       697
view_count        25861
Likes             23875
Dislikes           5590
count_max_view    25861
Year                  3
Month                12
MonYear              24
dtype: int64

In [19]:
Youtube_Unique_Video_Id.duplicated(subset=['Video ID'])

172       False
173       False
174       False
175       False
176       False
          ...  
143385    False
143386    False
143387    False
143388    False
143389    False
Length: 26360, dtype: bool

In [18]:
Reocurring_Trending_Videos = Youtube_Unique_Video_Id.groupby(['Video ID']).count().sort_values(by='title',ascending=False)
Reocurring_Trending_Videos.head(10)

,title,publishedAt,Channel ID,Channel Title,categoryId,trending_date,view_count,Likes,Dislikes,count_max_view,Year,Month,MonYear
Video ID,,,,,,,,,,,,,
69KrkMpvZdg,8,8,8,8,8,8,8,8,8,8,8,8,8
AWXvClaRtsI,7,7,7,7,7,7,7,7,7,7,7,7,7
hAxqygRdM4g,7,7,7,7,7,7,7,7,7,7,7,7,7
ZoPJVcHYlU0,6,6,6,6,6,6,6,6,6,6,6,6,6
BxOEj8ZeX2g,5,5,5,5,5,5,5,5,5,5,5,5,5
ifJYb2An7wE,5,5,5,5,5,5,5,5,5,5,5,5,5
Isim0ysZ6X4,5,5,5,5,5,5,5,5,5,5,5,5,5
kmk5vciFbek,4,4,4,4,4,4,4,4,4,4,4,4,4
Hb3rmh-_FMw,4,4,4,4,4,4,4,4,4,4,4,4,4


In [19]:
Youtube_Unique_Video_Id.loc[Youtube_Unique_Video_Id['Video ID'] == '69KrkMpvZdg']

,Video ID,title,publishedAt,Channel ID,Channel Title,categoryId,trending_date,view_count,Likes,Dislikes,count_max_view,Year,Month,MonYear
49396,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-20 00:00:00+00:00,0,0,0,0,2021,4,42021
49597,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-21 00:00:00+00:00,0,0,0,0,2021,4,42021
49810,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-22 00:00:00+00:00,0,0,0,0,2021,4,42021
50050,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-23 00:00:00+00:00,0,0,0,0,2021,4,42021
50283,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-24 00:00:00+00:00,0,0,0,0,2021,4,42021
50504,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-25 00:00:00+00:00,0,0,0,0,2021,4,42021
50735,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-26 00:00:00+00:00,0,0,0,0,2021,4,42021
50969,69KrkMpvZdg,Celebrating Luther Vandross's 70th Birthday,2021-04-19 15:00:08+00:00,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,1,2021-04-27 00:00:00+00:00,0,0,0,0,2021,4,42021


In [20]:
# display data frame first five rows
Youtube_Unique_Video_Id.head()

,Video ID,title,publishedAt,Channel ID,Channel Title,categoryId,trending_date,view_count,Likes,Dislikes,count_max_view,Year,Month,MonYear
172,cAtazIk1IYw,How To Make a Curried Egg Sandwich,2020-08-07 18:30:06+00:00,UCR4s1DE9J4DHzZYXMltSMAg,HowToBasic,26,2020-08-12 00:00:00+00:00,1238677,104736,3736,1238677,2020,8,82020
173,NYFHnIiA8gE,Cake Rescue Fixing Viral Cake Fails | How To C...,2020-08-07 09:30:04+00:00,UCsP7Bpw36J666Fct5M8u-ZA,How To Cook That,24,2020-08-12 00:00:00+00:00,938198,44088,565,938198,2020,8,82020
174,czwejgoH3zs,"Son, lemme teach you something new",2020-08-06 19:47:12+00:00,UCw03U5DZGLqvv5elJvXvR0Q,Bread Boys,24,2020-08-12 00:00:00+00:00,1722152,169501,927,1722152,2020,8,82020
175,dO6YihaqtaQ,Trump takes executive action to address econom...,2020-08-09 01:35:42+00:00,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,25,2020-08-12 00:00:00+00:00,1090847,10922,2517,1090847,2020,8,82020
176,sSjtGqRXQ9Y,JUDAS AND THE BLACK MESSIAH - Official Trailer,2020-08-06 23:01:42+00:00,UCjmJDM5pRKbUlVIzDYYWb6g,Warner Bros. Pictures,24,2020-08-12 00:00:00+00:00,971704,23311,1987,971704,2020,8,82020


In [21]:
# Basic Statistics for Data Frame
Youtube_Unique_Video_Id.describe().applymap('{:,.2f}'.format)

,categoryId,view_count,Likes,Dislikes,count_max_view,Year,Month
count,"26,062.00","26,062.00","26,062.00","26,062.00","26,062.00","26,062.00","26,062.00"
mean,18.85,"2,773,034.69","138,254.86","2,136.48","2,773,034.69","2,021.10",6.58
std,6.72,"6,780,496.24","394,301.76","11,102.08","6,780,496.24",0.70,3.44
min,1.00,0.00,0.00,0.00,0.00,"2,020.00",1.00
25%,17.00,"573,068.25","20,399.00",0.00,"573,068.25","2,021.00",4.00
50%,20.00,"1,174,506.00","49,130.50",395.00,"1,174,506.00","2,021.00",7.00
75%,24.00,"2,574,854.00","124,142.00","1,389.00","2,574,854.00","2,022.00",10.00
max,29.00,"264,407,389.00","16,021,534.00","879,354.00","264,407,389.00","2,022.00",12.00


In [22]:
#Getting category for the category file
#we need id and title from this file which are under items tag and snippet contain title
with open("US_category_id.json") as f:
    categories = json.load(f)["items"]
cat_dict = {}
category=[]
for cat in categories:
    category.append([cat["id"],cat["snippet"]["title"]])

In [23]:
df_category = pd.DataFrame(category,columns=['categoryId','category']) 
df_category.head(20)

,categoryId,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [24]:
df_category.categoryId=df_category.categoryId.astype('int64')
#merge YT Data Frame and df_category to get category names
df_merge = Youtube_Unique_Video_Id.merge(df_category, on = 'categoryId', how = 'inner')
df_merge.head()

,Video ID,title,publishedAt,Channel ID,Channel Title,categoryId,trending_date,view_count,Likes,Dislikes,count_max_view,Year,Month,MonYear,category
0,cAtazIk1IYw,How To Make a Curried Egg Sandwich,2020-08-07 18:30:06+00:00,UCR4s1DE9J4DHzZYXMltSMAg,HowToBasic,26,2020-08-12 00:00:00+00:00,1238677,104736,3736,1238677,2020,8,82020,Howto & Style
1,8r1D-vNUWsg,☁️ 🍞 CLOUD BREAD,2020-08-09 00:00:04+00:00,UCzqbfYjQmf9nLQPMxVgPhiA,emmymadeinjapan,26,2020-08-14 00:00:00+00:00,993107,41182,650,993107,2020,8,82020,Howto & Style
2,XlOLx3fyf10,TESTING TIKTOK ART TUTORIALS,2020-08-08 20:00:03+00:00,UCD4XIm3ZFhT72WjqhIXMN9w,LavenderTowne,26,2020-08-14 00:00:00+00:00,407183,42644,347,407183,2020,8,82020,Howto & Style
3,KnE6u4hFx14,PAINTING OUR FACES ON BRENT'S NEW HOUSE! (REVE...,2020-08-08 20:45:05+00:00,UCPpATKqmMV-CNRNWYaDUwiA,Alexa Rivera,26,2020-08-15 00:00:00+00:00,4782509,207021,3637,4782509,2020,8,82020,Howto & Style
4,YwJyis4yOEA,ANSWERING YOUR ASSUMPTIONS ABOUT US 😭,2020-08-09 13:46:28+00:00,UC21yq4sq8uxTcfgIxxyE9VQ,Carli Bybel,26,2020-08-15 00:00:00+00:00,350066,20393,379,350066,2020,8,82020,Howto & Style


In [33]:
# Which three video categories had the highest volume of videos posted? 

Top_3_category = df_merge.groupby(df_merge['category']).size().sort_values(ascending=False).head(3)

Top_3_category 


category
Entertainment    5165
Gaming           4959
Music            4219
dtype: int64

In [72]:
# creating dataframe with top_3_categories

Top_3_category_list = ['Entertainment','Gaming','Music']
All_rows_for_top_3_category = df_merge["category"].isin(Top_3_category_list)
All_rows_for_top_3_category_data = df_merge[All_rows_for_top_3_category]
All_rows_for_top_3_category_data.to_csv("output_data/Top_3_category_data.csv")


In [112]:
#len(df_merge)
#Top_3_category_details = df_merge.loc[df_merge.groupby(['category','MonYear']).sum('count_max_view').sort_values(by='count_max_view',ascending=False),['category','Month','Video ID','count_max_view']]
#Top_3_category_details
Top_3_category_details = df_merge.groupby(['category','MonYear']).sum('count_max_view').sort_values(by='count_max_view',ascending=False)
Top_3_category_details

categoryId  view_count     Likes  Dislikes  \
category              MonYear                                               
Music                 82020          2120  1436140019  90158358   3657904   
                      102020         2830  1413987047  86677089   1464187   
                      112020         2030  1287257055  79814011   1389204   
                      52021          1820  1263317139  73252201    942826   
Entertainment         52021          5472  1178513339  43021844   1324180   
...                                   ...         ...       ...       ...   
Travel & Events       32021            19      889109     29740       571   
Nonprofits & Activism 42021            29      831995     35559       254   
                      62021            29      457656     20828       223   
                      102021           29      387823     14755      2162   
Travel & Events       122020           19      266196      4156       168   

                               count_max_view    Year  Month  
category              MonYear                                 
Music                 82020        1436140019  428240   1696  
                      102020       1413987047  571660   2830  
                      112020       1287257055  410060   2233  
                      52021        1263317139  367822    910  
Entertainment         52021        1178513339  460788   1140  
...                                       ...     ...    ...  
Travel & Events       32021            889109    2021      3  
Nonprofits & Activism 42021            831995    2021      4  
                      62021            457656    2021      6  
                      102021           387823    2021     10  
Travel & Events       122020           266196    2020     12  

[344 rows x 7 columns]

In [113]:
Video_Category = Top_3_category_details[0]
View_Count = Top_3_category_details['view_count']
lineChart, = plt.plot(Video_Category, View_Count)

plt.title(title)
plt.xlabel("Category")
plt.ylabel("View Count")

#plt.savefig("output/line_plot.png")
plt.show()

KeyError: 0